In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import itertools
from tqdm import tqdm
from scipy import stats
from geolite2 import geolite2 
import string
%config InlineBackend.figure_format = 'retina' 

### Token flow

The goal of this notebook is to map token flow between addresses in the network. This notebook assumes that:
1. the actor relationship mining pipeline is successfully run (`main.py` produces the file `data/output.csv`)
2. user has a lookup table between miner IDs and addresses
3. user has a list of recent transactions

To generate (2) save the output of `select distinct id, address from id_addresses` (Lily schema) to a file.

To generate (3) save the output of `select * from parsed_messages where height > 22984802122800 and method='Send'` to a file.

In [188]:
worker_lookup_df = pd.read_csv('../data/worker_relationship_jun_20.csv', index_col=[0])

In [140]:
relationship_df = pd.read_csv('../data/output.csv', index_col=[0])
id_address_df = pd.read_csv('../data/id_lookup.csv', index_col=[0])
message_df = pd.read_csv('../data/recent_send_messages.csv', index_col=[0])

/Users/beng/miniforge3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### 0. Entity ID

As a starting point, we can use the pipeline output to establish an entity ID for every actor. This entity consists of workers, miners and owners. In the following we assemble two dicts / hash maps that assign an entity number and a country to every (or rather most) actors that we see.

In [141]:
entity_dict = {}
entity_location_dict = {}
for i in range(len(relationship_df)):
    
    # Convert the associated nodes into a list that we can iterate across
    nodes_in_entity = relationship_df.associated_nodes[i][1:-1].split(',')
    nodes_in_entity = [''.join(e for e in nie if e.isalnum()) for nie in nodes_in_entity]
    current_country = relationship_df.mode_country[i]

    for node in nodes_in_entity:
        entity_dict[node] = i
        entity_location_dict[node] = current_country

As an example

In [142]:
entity_dict['f01080779'], entity_location_dict['f01080779']

(0, 'CN')

### 1. Convert addresses to IDs for lookup

The messages use raw addresses, but our relationship list uses IDs. We therefore join on the ID-address lookup (2) so that every message has both addresses and IDs.

In [143]:
df = pd.merge(message_df, id_address_df, left_on = 'from_address', right_on='address')
df = df.rename(columns={'id': 'from_id'})
df = pd.merge(df, id_address_df, left_on = 'to_address', right_on='address')
df = df.rename(columns={'id': 'to_id'}).drop(columns=['address_x', 'address_y', 'height_in_month', 'etl_ts', 'cid'])
df.value = pd.to_numeric(df.value, errors='coerce')
df.value = df['value'] / 1e18

### 2. Map IDs to entities and countries

In [144]:
df['from_entity'] = df['from_id'].map(entity_dict).fillna('Unknown')
df['to_entity'] = df['from_id'].map(entity_dict).fillna('Unknown')
df['from_country'] = df['from_id'].map(entity_location_dict).fillna('Unknown')
df['to_country'] = df['from_id'].map(entity_location_dict).fillna('Unknown')

In [147]:
df[df.from_entity != 'Unknown']

,height,from_address,to_address,value,method,params,from_id,to_id,from_entity,to_entity,from_country,to_country
320,2298848,f3sicmx5zm4h44r2gqu57lmkj42hayeuf5cyvmfu43joxa...,f1kf7iuq6jntnifittghp4aqrjeicxpj72vl2l6za,108.00,Send,NaN,f01358402,f01694956,90.0,90.0,None,None
321,2317696,f3sicmx5zm4h44r2gqu57lmkj42hayeuf5cyvmfu43joxa...,f1kf7iuq6jntnifittghp4aqrjeicxpj72vl2l6za,535.00,Send,NaN,f01358402,f01694956,90.0,90.0,None,None
322,2307728,f3sicmx5zm4h44r2gqu57lmkj42hayeuf5cyvmfu43joxa...,f1kf7iuq6jntnifittghp4aqrjeicxpj72vl2l6za,177.00,Send,NaN,f01358402,f01694956,90.0,90.0,None,None
323,2312051,f3sicmx5zm4h44r2gqu57lmkj42hayeuf5cyvmfu43joxa...,f1kf7iuq6jntnifittghp4aqrjeicxpj72vl2l6za,103.00,Send,NaN,f01358402,f01694956,90.0,90.0,None,None
324,2308928,f3sicmx5zm4h44r2gqu57lmkj42hayeuf5cyvmfu43joxa...,f1kf7iuq6jntnifittghp4aqrjeicxpj72vl2l6za,115.00,Send,NaN,f01358402,f01694956,90.0,90.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
80196,2315951,f3un75jsptczswt7git7pqdpayw6dxwwuelotrj2t3g5c7...,f1aznindqy3bs7elop25zk3i552ts4upp76ggprbi,11.00,Send,NaN,f01621611,f01637416,259.0,259.0,CN,CN
80207,2312207,f16rbs365dhg5ycgihwihepuwyhwk33pu2ixm7zhi,f1pdibm3yaax3sklpiukv64ltuuvw4cdfcfbqcm5a,184.00,Send,NaN,f01631798,f01635593,156.0,156.0,None,None
80208,2312205,f16rbs365dhg5ycgihwihepuwyhwk33pu2ixm7zhi,f1yzrfjtvmvyqnld4527yuetvr2vzrq4iqjz4a2cy,16.00,Send,NaN,f01631798,f01937239,156.0,156.0,None,None
80210,2317351,f3r3ldadjxqj5xsr6xv6znybwmagpnz6ttipunss4jt3ex...,f1flqswmxaaplcslzggpf5m44zatzy6jeccysoqea,45.98,Send,NaN,f0845160,f01592763,125.0,125.0,None,None


### 3. Generate a graph

Text(0.5, 1.0, 'Entity 18 (CN) / miner (red), owner (green), worker (blue)')

In [191]:
entity_idx = 18
flows = df[df.from_entity == entity_idx][['from_id', 'to_id', 'value']].groupby(['from_id', 'to_id']).sum()

all_nodes_in_selected_entity = relationship_df.associated_nodes[entity_idx][1:-1].split(',')
all_nodes_in_selected_entity = [''.join(e for e in nie if e.isalnum()) for nie in all_nodes_in_selected_entity]

G = nx.DiGraph()
G.add_nodes_from(all_nodes_in_selected_entity)
for i in range(len(flows)):
    weight = np.round(flows.iloc[i].value, 1)
    from_id, to_id = flows.iloc[i].name
    G.add_edge(from_id, to_id, weight=weight)

color_map = []
for node in G.nodes:
    if node in worker_lookup_df.miner_id.values:
        color_map.append('C3')
    else:
        if node in worker_lookup_df.owner_id.values:
            color_map.append('C2')
        else:
            color_map.append('C0')

plt.figure(figsize=(8,6))
pos = nx.spring_layout(G)
grafo_labels = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_nodes(G, pos, node_color=color_map)
nx.draw_networkx_edges(G, pos)
edges_label = nx.draw_networkx_edge_labels(G, pos, edge_labels = grafo_labels)

for node, (x, y) in pos.items():
    plt.text(x, y-0.1, node, fontsize=8, ha='center', va='center')

plt.title('Entity %s (%s) / miner (red), owner (green), worker (blue)' % (entity_idx, stats.mode(df[df.from_entity == entity_idx].from_country.unique()).mode[0]))

Interestingly, there is no flow between the entities. There should be, in principle, some activity between a specific organisation and, say, an exchange. 